# Does negativity make success?

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import scipy

import warnings
warnings.filterwarnings("ignore")

#### Load the dataframe containing videos from 2019 and their features

This is a big file (860 MB) so we have stored it on Google Drive. Download it from the link below and storie it as `generated/2019/2019_videos_Typo_Emojis_NegWords_Sentiment_title_desc.parquet`. It was generated by the notebook `data_processing.ipynb`.

https://drive.google.com/file/d/1RmVSw2MBq0Ps0dwcTQjqZsDAuivXbUaZ/view?usp=share_link

In [ ]:
filepath = 'generated/2019/2019_videos_Typo_Emojis_NegWords_Sentiment_title_desc.parquet'
videos = pd.read_parquet(filepath, engine='fastparquet')

## What is negativity?

### Small intro (examples with sia from vader)

# Is there something to investigate ?

The trending videos dataset is used to determine a treshold on views and likes to split our dataset in two: Successful and not successful videos.

In [ ]:
#Loading of the trending datasets (US trending videos from 2018) [[put correct link/year]]
df_trends = pd.read_csv('data/additionnal/US_youtube_trending_data.csv')
df_trends

In [ ]:
#Determination of the tresholds
treshold_views = np.quantile(df_trends['view_count'], 0.4)
treshold_likes = np.quantile(df_trends['likes'], 0.4)

In [ ]:
#Dataset of the successful videos
videos_success = videos.loc[(videos['view_count'] > treshold_views)]
videos_success = videos_success.loc[(videos_success['like_count'] > treshold_likes)]

In [ ]:
#Dataset of the not successful videos
new_videos = videos.drop(videos_success.index)

The two dataset need to be modified in order to plot the comparison

In [ ]:
sia_neg_title_success = videos_success[['sia_negative_title', 'categories']]
sia_neg_description_success = videos_success[['sia_negative_description', 'categories']]


sia_neg_title = new_videos[['sia_negative_title', 'categories']]
sia_neg_description = new_videos[['sia_negative_description', 'categories']]

Construction of the negative title success dataset

In [ ]:
neg_title_suc = ['Negative sentiment in title']
sia_bool_neg_title_suc = ['successful']

neg_title_suc = neg_title_suc*len(sia_neg_title_success)
sia_bool_neg_title_suc = sia_bool_neg_title_suc*len(sia_neg_title_success)
neg_title_suc_cat = sia_neg_title_success['categories']
sia_neg_title_success = sia_neg_title_success['sia_negative_title']

In [ ]:
sia_neg_title_success_values = sia_neg_title_success.values
sia_neg_title_success = sia_neg_title_success_values.tolist()

In [ ]:
#Creation of the negative title success dataset
data_given = {'Type of sentiment': neg_title_suc, 'SIA value': sia_neg_title_success, "Video's succes": sia_bool_neg_title_suc, 'categories':neg_title_suc_cat}

neg_title_suc_df = pd.DataFrame(data = data_given)

Construction of the negative title not success dataset

In [ ]:
neg_title = ['Negative sentiment in title']
sia_bool_neg_title = ['not successful']

neg_title = neg_title*len(sia_neg_title)
sia_bool_neg_title = sia_bool_neg_title*len(sia_neg_title)
neg_title_cat = sia_neg_title['categories']
sia_neg_title = sia_neg_title['sia_negative_title']

In [ ]:
sia_neg_title_values = sia_neg_title.values
sia_neg_title = sia_neg_title_values.tolist()


In [ ]:
#Creation of the negative title success dataset
data_given = {'Type of sentiment': neg_title, 'SIA value': sia_neg_title, "Video's succes": sia_bool_neg_title, 'categories':neg_title_cat}

neg_title_df = pd.DataFrame(data = data_given)



In [ ]:
#Total dataset for the title (merge success and not)
neg_title_total_df = neg_title_suc_df.append(neg_title_df, ignore_index=True)

Construction of the negative description success dataset

In [ ]:
neg_description_suc = ['Negative sentiment in description']
sia_bool_neg_description_suc = ['successful']

neg_description_suc = neg_description_suc*len(sia_neg_description_success)
sia_bool_neg_description_suc = sia_bool_neg_description_suc*len(sia_neg_description_success)
neg_description_suc_cat = sia_neg_description_success['categories']
sia_neg_description_success = sia_neg_description_success['sia_negative_description']

In [ ]:
sia_neg_description_success_values = sia_neg_description_success.values
sia_neg_description_success = sia_neg_description_success_values.tolist()

In [ ]:
#Creation of the negative description success dataset
data_given = {'Type of sentiment': neg_description_suc, 'SIA value': sia_neg_description_success, "Video's succes": sia_bool_neg_description_suc, 'categories':neg_description_suc_cat}

neg_description_suc_df = pd.DataFrame(data = data_given)


Construction of the negative description not success dataset

In [ ]:
neg_description = ['Negative sentiment in description']
sia_bool_neg_description = ['not successful']

neg_description = neg_description*len(sia_neg_description)
sia_bool_neg_description = sia_bool_neg_description*len(sia_neg_description)



In [ ]:
neg_description_cat = sia_neg_description['categories']
sia_neg_description = sia_neg_description['sia_negative_description']

In [ ]:
sia_neg_description_values = sia_neg_description.values
sia_neg_description = sia_neg_description_values.tolist()

In [ ]:
#Creation of the negative description not success dataset
data_given = {'Type of sentiment': neg_description, 'SIA value': sia_neg_description, "Video's succes": sia_bool_neg_description, 'categories':neg_description_cat}

neg_description_df = pd.DataFrame(data = data_given)


#Total dataset for the description (merge success and not)
neg_description_total_df = neg_description_suc_df.append(neg_description_df, ignore_index=True)

#Creation of the complete dataset to be plot
neg_total_df = neg_title_total_df.append(neg_description_total_df, ignore_index=True)


In [ ]:
sns.barplot(data=neg_total_df, x="Type of sentiment", y="SIA value", hue="Video's succes", color = 'blue', palette = 'hls')
plt.title('Difference in the SIA value in the title depending on the succes of the video')
plt.show()

## Regression analysis [[Djian]]

### Overall

try description and title (look at R to find what is best) [[Djian: description is better]]

In [ ]:
def print_regression(data, formula):
    model = smf.ols(formula=formula, data=data)
    np.random.seed(2)
    results = model.fit()
    print(results.summary())

In [ ]:
# Remove videos where 'like_count' is NaN
videos = videos[videos['like_count'].isna() == False]

# Convert some rows to float
videos['like_count'] = videos['like_count'].astype(float)
videos['dislike_count'] = videos['dislike_count'].astype(float)
videos['view_count'] = videos['view_count'].astype(float)

# New columns for the log of the counts (+1 so that the log is always defined)
videos['log_view_count'] = np.log(videos['view_count'] + 1)
videos['log_like_count'] = np.log(videos['like_count'] + 1)
videos['log_dislike_count'] = np.log(videos['dislike_count'] + 1)

In [ ]:
success_factors = ['log_view_count', 'log_like_count', 'log_dislike_count']


def regression_formula(success_factor):
    f = f'{success_factor} ~ '
    f += 'sia_negative_description ' 
    f += '+ sia_positive_description '
    f += '+ sia_neutral_description '
    return f


formulas = [regression_formula(s) for s in success_factors]

for f in formulas:
    print(f'Regression analysis for formula \n{f}')
    print_regression(data=videos, formula=f)
    print('')
    print('')

### By category

In [ ]:
# Find the categories
categories = set(videos['categories'].values)
categories.remove(None)

In [ ]:
regression_for_success_factor = dict()

for success_factor in success_factors:
    f = regression_formula(success_factor)
    
    results_params_f = dict()

    for category in categories:
        videos_category = videos[videos['categories'] == category]
        model = smf.ols(formula=f, data=videos_category)
        np.random.seed(2)
        results = model.fit()
        results_params_f[category] = pd.concat([results.params, results.pvalues], keys=['parameter', 'p-value'])

    df_regression = pd.DataFrame(results_params_f).transpose()
    
    regression_for_success_factor[success_factor] = df_regression

In [ ]:
def plot_regression(df_regression):
    
    df_reg = df_regression.copy()
    
    # Drop p-values and `Intercept`, remove index
    df_reg = df_reg['parameter']
    df_reg = df_reg.drop('Intercept', axis=1)
    df_reg = df_reg.reset_index()

    # Plot
    fig, ax = plt.subplots(figsize=(9, 6))
    plt.scatter(x=df_reg['index'], y=df_reg['sia_negative_description'], marker='$:($', color='crimson', s=50)
    plt.scatter(x=df_reg['index'], y=df_reg['sia_neutral_description'],  marker='$:|$', color='gray', s=50)
    plt.scatter(x=df_reg['index'], y=df_reg['sia_positive_description'],  marker='$:)$', color='dodgerblue', s=50)
    plt.xticks(rotation=90)
    plt.ylabel('log_count')
    plt.show()

In [ ]:
# Plot the regression parameters for various success factors

for success_f in success_factors:
    print(f'Linear regression for {success_f}')
    plot_regression(regression_for_success_factor[success_f])
    print('')

In [ ]:
# Alternative: seaborn plot
'''
# Colors for the plot
palette_sentiment = {
    'sia_negative_description': 'crimson',
    'sia_neutral_description': 'gray',
    'sia_positive_description': 'dodgerblue',
    'Intercept': 'black'

}

# Drop p-values and `Intercept`, remove index
df_regression = df_regression['parameter']
df_regression = df_regression.drop('Intercept', axis=1)
df_regression = df_regression.reset_index()

# Convert the dataframe to long form, for seaborn plot
df_regression_melt = df_regression.melt('index', var_name='sentiment_type', value_name='sentiment_value')

# Plot
sns.scatterplot(
    data=df_regression_melt, 
    x='index', 
    y='sentiment_value', 
    hue='sentiment_type',
    s=40,
    marker='D',
    palette=palette_sentiment,
    #aspect=2,
    #jitter=False,  # for vertically aligned datapoints in each category
    alpha=0.8
)
plt.xticks(rotation=90)
plt.show()
'''
' '

Creation of the interactive plot for the log view count depending on the sia

In [ ]:
df_reg = regression_for_success_factor['log_view_count'].copy()
    
# Drop p-values and `Intercept`, remove index
df_reg = df_reg['parameter']
df_reg = df_reg.drop('Intercept', axis=1)
df_reg = df_reg.reset_index()



In [ ]:
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objs as go
import plotly.express as px
import dash
import math
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

In [ ]:
app = JupyterDash(__name__)


app.layout = html.Div([
    html.H4('Difference in the SIA value in the title depending on the succes of the video'),
    dcc.Dropdown(
        id="dropdown",
        options=['People & Blogs', 'Science & Technology', 'Gaming',
       'Film & Animation', 'News & Politics', 'Howto & Style', 'Music',
       'Entertainment', 'Education', 'Comedy', 'Travel & Events',
       'Sports', 'Autos & Vehicles', 'Pets & Animals',
       'Nonprofits & Activism'],
        value="People & Blogs",
        clearable=False,
    ),
    dcc.Graph(id="graph"),
])


@app.callback(
    Output("graph", "figure"), 
    Input("dropdown", "value"))
def update_bar_chart(cat):
    df = df_reg # replace with your own data source
    
    mask = df["index"] == cat
    
    inter = df_reg[mask]
    inter_dic = {'Sentiment': ['sia_negative_description', 'sia_positive_description', 'sia_neutral_description'], 'Value': [inter['sia_negative_description'].values[0], inter['sia_positive_description'].values[0], inter['sia_neutral_description'].values[0]]}
    inter_df = pd.DataFrame(data= inter_dic)

    fig = px.bar(inter_df, x="Sentiment", y="Value", barmode='group')
    return fig


app.run_server(debug = True,port=8050)

## Evolution of channels with negativity [[Victor]]

## What does successful negativity look like?

### Most used words: the vocabulary of videos that are negative and successful (for different categories) [[Maj]]

Make 'histogram' of words in title/desc for videos that are very negative and have lots of success (maybe do it for each category). Example:

https://ldrame21.github.io/metoo-media-impact/#data-story-title

### Try to extract topics from negative and successful videos